# 'usdeaths' is a monthly data which shows the number of accidental deaths in the USA from January 1973 to December 1978.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
usdeaths = pd.read_csv('../input/us-accidental-death/usdeaths.csv')
usdeaths.head()

In [3]:
usdeaths.drop(['Unnamed: 0'], axis=1, inplace=True)

cols = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
usdeaths_reshaped = pd.DataFrame(usdeaths.values.reshape(-1, 12), 
                         columns=cols,                        # Month 
                         index=range(1973, 1979)) 

usdeaths_reshaped

Use exponential smoothing to predict the accidental deaths in the US for January 1978, based on the historical data from January 1973 to December 1977. Plot the predicted data along with the actual and the past data.

# Importing libraries

In [4]:
import numpy as np

# Exponential smoothing is calculated and rounded off to 2 decimal values

In [5]:
np.round(usdeaths.ewm(alpha = 0.2, adjust = False).mean().head(-10), 2)

# Simple exponential smoothing
Finding alpha value for the forecasted value and mean squared error

In [6]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.metrics import mean_squared_error
for i in range(1, 11):
    model = SimpleExpSmoothing(usdeaths.iloc[:61]).fit(smoothing_level=i/10, optimized=False)
    forecasted_val = np.round(model.forecast(1), 2)
    print('alpha = ', i/10, '| Forecasted value: ', forecasted_val, 
          '| MSE: ', np.round(mean_squared_error(np.array(usdeaths.iloc[61]), np.array([forecasted_val])), 2))

# Importing libraries

In [7]:
import numpy as np
import scipy as sp

# Function to find optimum value of alpha

In [8]:
def optimum_alpha(x):
    model = SimpleExpSmoothing(usdeaths.iloc[:61]).fit(smoothing_level=x, optimized = False)
    forecasted_val = np.round(model.forecast(1), 2)
    mse = np.round(mean_squared_error(np.array(usdeaths.iloc[61]), np.array([forecasted_val])), 2)
    print('alpha: ', np.round(x[0], 5), 'MSE: ', mse)
    return mse
optimum_alpha_result = sp.optimize.fmin(optimum_alpha, x0=1)
if optimum_alpha_result < 0:
    optimum_alpha_result = 0.001 # Least value, you can perform further optimization to improve it
optimum_alpha_result

Use ARIMA to predict the monthly deaths in the US for the year 1978, based on the historical data from January 1973 to December 1977. Also, plot the forecasted data along with the actual and the past data.

Important document: https://docs.w3cub.com/statsmodels/generated/statsmodels.tsa.statespace.sarimax.sarimaxresults.conf_int

# Importing libraries

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Values to keep in training data, k

In [10]:
k = 60

# Model building

In [11]:
model = SARIMAX(np.log(usdeaths.iloc[:k]), 
                order=(2, 1, 2), 
                seasonal_order=(1, 1, 1, 12), 
                enforce_stationarity=False, 
                enforce_invertibility=False)
model_fit = model.fit(disp=False)
model_conf = model_fit.conf_int()
print(model_conf)

# Visualization

In [12]:
plt.figure(figsize=(15, 5))
plt.plot(usdeaths.iloc[:k + 1], label='Train data')
plt.plot(usdeaths.iloc[k:], label='Test data')
plt.plot(np.round(np.exp(model_fit.forecast(72-k))), label='Forecasted data')
plt.legend(loc='upper center')
sns.despine()
plt.xlabel('Year', fontsize=15)
plt.ylabel('Number of Deaths', fontsize=15)
plt.title('SARIMA forecasting', fontsize=18, weight='bold')
plt.show()

# Finding the mean squared error

In [13]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(usdeaths.iloc[k:], np.round(np.exp(model_fit.forecast(72-k)))))

# Importing libraries

In [14]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Iterating over parameters

In [15]:
rmse = []
params = []
for p in range(3):
    for d in range(2):
        for q in range(3):
            for P in range(3):
                for D in range(2):
                    for Q in range(3):            
                        try:
                            model = SARIMAX(np.log(usdeaths.iloc[:k]), 
                                            order=(p, d, q), 
                                            seasonal_order=(P, D, Q, 12), 
                                            enforce_stationarity=False, 
                                            enforce_invertibility=False)
                            model_fit = model.fit(disp=False)                            
                            rmse.append(np.sqrt(mean_squared_error(usdeaths.iloc[k:], 
                                                np.round(np.exp(model_fit.forecast(72-k))))))
                            params.append(((p, d, q), (P, D, Q)))
                        except:
                            pass
                        
# Storing RMSE and parameters and sorting the dataframe based on RMSE in ascending order
res = pd.DataFrame([rmse, params]).T.sort_values([0])
res.columns = ['RMSE', 'Params']
res.head(1)

In [16]:
plt.figure(figsize=(15, 5))
plt.plot(rmse, label='All RMSEs')
plt.scatter(np.arange(210, 211), res.iloc[0, 0], s=70, c='g', label='Chosen RMSE')
plt.legend()
plt.xlabel('Iterations', fontsize=15)
plt.ylabel('RMSE', fontsize=15)
plt.title('RMSE values across all possible parameter combinations', weight='bold', fontsize=18)
plt.show()